# grab IMERG fields for the bounding box +/- 5 degrees and write out 


In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import geopandas as gp

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

## Read all lakes in 2014-2018 
#### lakes dataframe was "improved" in LakeCaseStudy.ipynb with firsttime added

In [3]:
df = pd.read_csv('ccvls_stats_2014-2018.improved.csv')
# df

In [4]:
equatorcases = df[ abs(df.coastlat) < 10 ] # 162 of them exceeding 1 day 
eq7cases = equatorcases[ equatorcases.dur_days > 6 ]  # 39 of them exceeding a week (more than 6 in days) 
eq7cases.maxarea.size

39

# Grab IMERG for the event bounding box

In [5]:
IMERG = xr.open_dataset('https://gpm1.gesdisc.eosdis.nasa.gov:443/dods/GPM_3IMERGHH_06')
IMERG

<xarray.Dataset>
Dimensions:   (time: 374016, lat: 1800, lon: 3600)
Coordinates:
  * time      (time) datetime64[ns] 2000-06-01 ... 2021-09-30T23:30:00
  * lat       (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon       (lon) float64 -179.9 -179.8 -179.8 -179.7 ... 179.8 179.8 179.9
Data variables:
    hqprecip  (time, lat, lon) float32 ...
    precipc   (time, lat, lon) float32 ...
Attributes:
    title:        GPM IMERG Final Precipitation L3 Half Hourly 0.1 by 0.1 deg...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Sat Jun 17 07:42:41 GMT 2023 : imported by GrADS Data Serve...

In [6]:
pchourly = IMERG.precipc[::2,:,:]
pchourly

<xarray.DataArray 'precipc' (time: 187008, lat: 1800, lon: 3600)>
[1211811840000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 ... 2021-09-30T23:00:00
  * lat      (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float64 -179.9 -179.8 -179.8 -179.7 ... 179.8 179.8 179.9
Attributes:
    long_name:  precipitationcal [mm/hr]

In [7]:
!date

Sat Jun 17 02:03:20 MDT 2023


In [ ]:
for icase in range(len(eq7cases)): 

# Select cases by small to large areatime
    case = eq7cases.sort_values('areatime').iloc[icase]
    filename = case.filename
    print(icase, ' case, areatime ',filename, case.areatime)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = pchourly.sel(lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                time=slice(gdf.time.min(),gdf.time.max())             )

    d2.to_netcdf('~/Box/VaporLakes/data/LAKEBYLAKE/IMERG/'+filename[0:-8]+'_IMERG_precipc_hourly.nc')
    

0  case, areatime  2018_11_20_11_lat4p616S.geojson 1892.647086291892


In [ ]:
!date